In [1]:
import torch
from torch import nn
from torchvision import models as torch_models
import torchvision.transforms as transforms
import numpy as np
import os
from PIL import Image
from skimage import io
from tqdm import tqdm
import pandas as pd
import argparse
import time

/home/wangyh/miniconda3/envs/patho_AI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
data_df = np.load('../config/data_segmentation_csv/10X_full.npy',allow_pickle=True).item()
test_df = data_df['full_list']

In [15]:
test_df.head()
len(test_df)

363

In [2]:
from model.resnet_simclr import ResNetSimCLR
import torch

In [3]:
state_dict = torch.load('simclr_feature_extractor/runs/Apr20_10-20-33_ubuntu-T640/checkpoints/model.pth')
state_dict = {k.replace('module.',''):v for k,v in state_dict.items()}
model = ResNetSimCLR(base_model='resnet50',out_dim=512)
model.load_state_dict(state_dict)
model = model.eval()
model.cuda()

/home/wangyh/miniconda3/envs/patho_AI/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/wangyh/miniconda3/envs/patho_AI/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Feature extractor: resnet50


ResNetSimCLR(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2

In [20]:
# img_paths = test_df['img_list'].iloc[5]
# imgs = []

data_transform = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# for i in tqdm(img_paths):
#     image = Image.open(i).convert('RGB')
#     image_tensor = data_transform(image).unsqueeze(0)
#     imgs.append(image_tensor)
# input = torch.cat(imgs, dim=0)

In [21]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

#### 单个slide feature extraction测试:通过指定i（iloc，not index）切换不同slides

In [18]:
# i=202
del i
batch_size = 64
# slide_features = {}
# with torch.no_grad():
#     paths = test_df['img_list'].iloc[i]
#     N_tumor_patch = len(paths)
#     feature_list = []
#     for batch_idx in tqdm(range(0, N_tumor_patch, batch_size)):
#         end = batch_idx + batch_size if batch_idx+batch_size < N_tumor_patch else N_tumor_patch
#         batch_paths = paths[batch_idx: end]
#         images = []
#         for p in batch_paths:
#             image = Image.open(p).convert('RGB')
#             image_tensor = data_transform(image).unsqueeze(0)
#             images.append(image_tensor)
#         images = torch.cat(images, dim=0)

#         features = model(images.cuda())[1]
#         if len(features.shape) == 1:
#             features = features.unsqueeze(0)
#         feature_list.append(features.detach().cpu())
#         del features

#     feature_list = torch.cat(feature_list, dim=0)
#     slide_features[f'index{i}'] = (test_df['dir_uuid'].iloc[i],test_df['TMB_H/L'].iloc[i],feature_list)

In [106]:
# slide_features[f'index{i}'][2].shape

torch.Size([129, 512])

In [27]:
import numpy as np
s_features = np.load('../data/10X_full_slide_features_res50.npy',allow_pickle=True).item()  #未增加上面cell中关于维度处理时提取的无问题slides
unextracted = np.load('../data/10X_full_not_extracted_res50.npy',allow_pickle=True) #有问题的slides的列表


In [28]:
s_features['index98']
# unextracted

('bc09ee8e-ff24-4635-852f-84cabce80c0f',
 'L',
 tensor([[-0.0237,  0.0373, -0.0691,  ..., -0.2615, -0.1424,  0.0990],
         [ 0.0070, -0.0482,  0.1075,  ..., -0.1204, -0.0039, -0.1455],
         [ 0.1838, -0.0326,  0.1583,  ..., -0.1497, -0.0781, -0.0553],
         [-0.0888,  0.1956, -0.2567,  ...,  0.0945,  0.1830, -0.0325],
         [-0.3277, -0.1676, -0.2626,  ..., -0.2340,  0.0220, -0.1692]]))

# slide feature extraction 拯救计划：补充未被提取的图片们

#### 多个slide feature extraction测试:通过指定i——list（iloc，not index）切换不同slides——series

In [22]:
for i in tqdm(unextracted):
    try:
        paths = test_df['img_list'].iloc[i]
        N_tumor_patch = len(paths)
        feature_list = []
        for batch_idx in range(0, N_tumor_patch, batch_size):
            end = batch_idx + batch_size if batch_idx+batch_size < N_tumor_patch else N_tumor_patch
            batch_paths = paths[batch_idx: end]
            images = []
            for p in batch_paths:
                image = Image.open(p).convert('RGB')
                image_tensor = data_transform(image).unsqueeze(0)
                images.append(image_tensor)
            images = torch.cat(images, dim=0)

            features = model(images.cuda())[1]
            if len(features.shape) == 1:
                features = features.unsqueeze(0)
            feature_list.append(features.detach().cpu())
            del features

        feature_list = torch.cat(feature_list, dim=0)
        s_features[f'index{i}'] = (test_df['dir_uuid'].iloc[i],test_df['TMB_H/L'].iloc[i],feature_list)
    except Exception as e:
        print(f'wrong in slide{i}, error as {e}')
#         unextracted.append(i)

100%|██████████| 8/8 [00:12<00:00,  1.55s/it]


In [23]:
len(s_features)

363

In [24]:
s_features['index98']

('bc09ee8e-ff24-4635-852f-84cabce80c0f',
 'L',
 tensor([[-0.0237,  0.0373, -0.0691,  ..., -0.2615, -0.1424,  0.0990],
         [ 0.0070, -0.0482,  0.1075,  ..., -0.1204, -0.0039, -0.1455],
         [ 0.1838, -0.0326,  0.1583,  ..., -0.1497, -0.0781, -0.0553],
         [-0.0888,  0.1956, -0.2567,  ...,  0.0945,  0.1830, -0.0325],
         [-0.3277, -0.1676, -0.2626,  ..., -0.2340,  0.0220, -0.1692]]))

用完全版的slide features覆盖之前的不完全版

In [26]:
# np.save('../data/10X_full_slide_features_res50.npy',s_features)